<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/gpts/xplainT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch
import torch.nn as nn

In [3]:
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-04 16:18:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2025-07-04 16:18:33 (116 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open(mode="r", file="input.txt") as f:
  text = f.read()

text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [14]:
vocab = sorted(set(list(text)))

In [21]:
stoi = {v : k for k, v in enumerate(vocab)}
itos = {k : v for k, v in enumerate(vocab)}

encode = lambda seq : [stoi[s] for s in seq]
decode = lambda idx : "".join([itos[x] for x in idx])

In [28]:
data = torch.tensor(encode(text))

In [26]:
###################
# HYPERPARAMETERS #
###################
embed_dim = 32
block_size = 8

In [29]:
embedding = nn.Embedding(embed_dim, embed_dim)

In [43]:
embedding(torch.randint(3, (1,2)))

tensor([[[ 1.0869e+00,  7.8041e-02,  7.9894e-01,  1.0715e+00,  9.5654e-01,
           7.6049e-01,  1.7241e-01,  4.7712e-01,  4.1163e-01, -1.7514e+00,
          -4.7344e-01, -2.0426e+00,  3.2502e-01,  2.1115e+00,  9.8810e-01,
           9.5602e-01, -4.8398e-01, -1.0437e+00,  2.2327e-01, -2.7945e-01,
           4.7489e-01, -5.9326e-01,  1.8130e-01, -1.9923e+00,  8.7360e-01,
           4.6991e-02, -7.4626e-01,  1.2625e+00,  8.5126e-01,  1.5300e+00,
           1.5616e+00, -2.7176e-01],
         [ 1.7934e+00, -2.0259e+00, -2.1625e-01,  8.7295e-01, -3.0981e-01,
          -1.2320e+00, -1.5905e-02, -1.0086e+00,  1.9879e+00, -1.0159e+00,
           1.1537e+00,  2.4487e+00, -2.1411e-01,  2.1464e+00, -5.1017e-01,
          -2.1689e-01,  2.7450e+00, -1.1084e+00, -3.4781e-01, -5.8417e-01,
           4.7171e-01,  3.7252e-01, -1.0554e+00,  1.1568e+00, -1.2528e-01,
          -4.2954e-01, -8.9155e-01, -1.8819e+00, -7.0897e-01, -1.3370e+00,
          -8.6959e-01,  2.7468e-04]]], grad_fn=<EmbeddingBackwa